In [1]:
import os
import numpy as np
import timeit
import pyspark
import pandas as pd
import s3fs
import time
import ast
import operator
from datetime import datetime
import statistics

In [2]:
sc = pyspark.SparkContext.getOrCreate()
ss = pyspark.sql.SparkSession.builder.getOrCreate()

## Set up the data address book

In [3]:
s3_address = "s3a://msds630-kaggle-competition/"
dataset_addr_book = {}
dataset_name_list = ["events", "messages", "attributes", "sessions"]
for name in dataset_name_list:
    dataset_addr_book[name] = "".join([s3_address, name, ".csv"])

dataset_addr_book

{'attributes': 's3a://msds630-kaggle-competition/attributes.csv',
 'events': 's3a://msds630-kaggle-competition/events.csv',
 'messages': 's3a://msds630-kaggle-competition/messages.csv',
 'sessions': 's3a://msds630-kaggle-competition/sessions.csv'}

## Load Sessions

In [4]:
sessions_rdd = sc.textFile(dataset_addr_book["sessions"])\
               .map(lambda line : line.encode('ascii', 'ignore'))
cols_sessions = sessions_rdd.map(lambda x: x.split(',')).take(1)[0]
print(cols_sessions)
print(len(cols_sessions))

['app_id', 'session_id', 'start_timestamp', 'timezone', 'timezone_offset', 'previous_sessions_duration', 'user_created_timestamp', 'is_user_first_session', 'is_session', 'is_developer', 'is_wau', 'is_mau', 'country', 'region', 'city', 'latitude', 'longitude', 'locale', 'os_name', 'session_index', 'device_id', 'user_id_hash']
22


In [17]:
['app_id', 'session_id', 'start_timestamp', 'timezone', 'timezone_offset', 'previous_sessions_duration', 'user_created_timestamp', 'is_user_first_session', 'is_session', 'is_developer', 'is_wau', 'is_mau', 'country', 'region', 'city', 'latitude', 'longitude', 'locale', 'os_name', 'session_index', 'device_id', 'user_id_hash'][8
]

'is_session'

In [5]:
sessions_data = sessions_rdd.filter(lambda x: 'app_id' not in x)\
                            .map(lambda x: x.split(','))
sessions_data.take(1)

[['4724682771660800',
  '5558845121177764917',
  '1542215364580',
  'Asia/Manila',
  '28800000',
  '25837591',
  '1538874289458',
  'false',
  'true',
  'false',
  'false',
  'false',
  'PH',
  '00',
  'makati',
  '14.554729461669922',
  '121.02444458007812',
  'en_GB',
  'Android OS',
  '30',
  '546a3d98-d540-4e72-ad82-9ebd64e0839b',
  '9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006']]

In [20]:
sessions_data_adj = sessions_data.filter(lambda x: x[8] == 'true')\
                                 .filter(lambda x: x[10] == 'false')\
                                 .filter(lambda x: x[11] == 'false')\
                                 .filter(lambda x: x[9] == 'false')
sessions_data_adj.take(1)

[['4724682771660800',
  '5558845121177764917',
  '1542215364580',
  'Asia/Manila',
  '28800000',
  '25837591',
  '1538874289458',
  'false',
  'true',
  'false',
  'false',
  'false',
  'PH',
  '00',
  'makati',
  '14.554729461669922',
  '121.02444458007812',
  'en_GB',
  'Android OS',
  '30',
  '546a3d98-d540-4e72-ad82-9ebd64e0839b',
  '9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006']]

In [25]:
sessions_data_adj1 = sessions_data_adj.map(lambda x: [x[-1], x[1], x[6], x[12], x[18], x[20]])
sessions_data_adj1.take(1)

[['9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006',
  '5558845121177764917',
  '1538874289458',
  'PH',
  'Android OS',
  '546a3d98-d540-4e72-ad82-9ebd64e0839b']]

In [26]:
temp_pair_dict = {}
for i in range(6):
    if i != 0:
        temp_pair = sessions_data_adj1.map(lambda x: (x[0], x[i]))\
                                      .groupByKey().map(lambda x: [x[0], list(x[1])])
        temp_pair_dict[i] = temp_pair

In [27]:
sessions_data_adj2 = temp_pair_dict[1]
sessions_data_adj2 = sessions_data_adj2.leftOuterJoin(temp_pair_dict[2])\
                                             .map(lambda x: (x[0], list(x[1])))
sessions_data_adj2 = sessions_data_adj2.leftOuterJoin(temp_pair_dict[3])\
                                             .map(lambda x: (x[0], list(x[1])))\
                                             .map(lambda x: (x[0], [x[1][0][0], x[1][0][1], x[1][1]]))
sessions_data_adj2 = sessions_data_adj2.leftOuterJoin(temp_pair_dict[4])\
                                             .map(lambda x: (x[0], list(x[1])))\
                                             .map(lambda x: (x[0], [x[1][0][0], x[1][0][1], x[1][0][2], x[1][1]]))
sessions_data_adj2 = sessions_data_adj2.leftOuterJoin(temp_pair_dict[5])\
                                             .map(lambda x: (x[0], list(x[1])))\
                                             .map(lambda x: (x[0], [x[1][0][0], x[1][0][1], x[1][0][2], 
                                                                    x[1][0][3], x[1][1]]))
sessions_data_adj2.take(1)

[('46d54b4fab292c461cdf8f3825f7106907b300b2d28cf3bb9fff11df774cd6f7',
  [['3771536717571927006'],
   ['1539986698775'],
   ['US'],
   ['iOS'],
   ['443AE602-C493-40D2-A125-4245B72AC4AA']])]

In [28]:
def most_common(lst):
    return max(set(lst), key=lst.count)

In [29]:
sessions_data_adj3 = sessions_data_adj2.map(lambda x: [x[0],
                                                       most_common(x[1][1]),
                                                       most_common(x[1][2]),
                                                       most_common(x[1][3]),
                                                       len(set(x[1][4]))])
sessions_data_adj3.take(1)

[['46d54b4fab292c461cdf8f3825f7106907b300b2d28cf3bb9fff11df774cd6f7',
  '1539986698775',
  'US',
  'iOS',
  1]]

In [31]:
sessions_data_adj3.filter(lambda x: x[2] == "NA").take(1)

[['4c11dfd3e42377b2c79ebb6db9189947ce1400406129101cebcf5e4f2c7c1764',
  '1538661759993',
  'NA',
  'Android OS',
  1]]

In [37]:
def convertTime(ts_str):
    return datetime.utcfromtimestamp(int(ts_str)/1000.0).strftime('%Y-%m-%d %H:%M:%S')
convertTime('1538661759993')

'2018-10-04 14:02:39'

In [39]:
sessions_data_adj3.filter(lambda x:x[1] == "").take(1)

[]

In [41]:
sessions_data_adj4 = sessions_data_adj3.map(lambda x: [x[0], convertTime(x[1]),
                                                       'NAM' if x[2] == "NA" else x[2],
                                                       x[3], x[4]])
sessions_data_adj4.take(1)

[['46d54b4fab292c461cdf8f3825f7106907b300b2d28cf3bb9fff11df774cd6f7',
  '2018-10-19 22:04:58',
  'US',
  'iOS',
  1]]

## Data Dictionary
0 - user_id  
1 - date_user_created   
2 - most_freq_country  
3 - most_freq_os  
4 - num_uniq_device_id

In [42]:
sessions_data_adj5 = sessions_data_adj4.map(lambda x: ','.join([str(y) for y in x]))
sessions_data_adj5.take(1)

['46d54b4fab292c461cdf8f3825f7106907b300b2d28cf3bb9fff11df774cd6f7,2018-10-19 22:04:58,US,iOS,1']

In [43]:
with open('sessions.csv', 'w') as f:
    for line in sessions_data_adj5.collect():
        f.write(line)
        f.write('\n')

In [44]:
sessions_data_adj5.count()

619519